# Port Performance Index Project - Metrics Development Workbook

This workbook develops port performance metrics using the AIS data processed in 'ais_ingest.ipynb' in the 'ais data' folderm, along with port and dock data from the BTS and USACE. 

In [3]:
#prelims
import polars as pl
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=50);


## Load and Pre-process Data 

### AIS Data

The vessel locations and status (e.g., "under way", "anchored", "moored") data include all AIS messages; for the purposes of the PPI, we only need to know when a vessel *changes* status, so all other observations are dropped. 

Early exploratory analysis identified that vessel AIS statuses often change for very short periods of time, likely due to single AIS pings of errant or missing status. For example, a ship at berth for 4 hours may occaisionally ping "under power" even though the vessel remained at berth. To correct for this, we drop very short duration statuses where the previous and next statuses match.  

In [4]:
#set minimum meaningful status duration (minutes)
min_duration = 10

In [5]:
#init list of lazyframes
lfs = []
#process each parquet file individually into lazyframes
for file in glob.glob('ais data/data/ais_clean/*.parquet'):
    try:
        #check file integrity 
        pl.scan_parquet(file).collect_schema()
        #read file
        lf = (
            pl.scan_parquet(file)
            #drop smaller vessels
            .filter(pl.col('length')>100)
            #sort by vessel and time
            .sort(['mmsi', 'time'])
            #drop messages from the same vessel with same timestamp
            .unique(subset=['mmsi', 'time'])
            #indicate whether status is the same as previous row (Fill value needed to avoid status 0 evaluating as equal to false)
            .with_columns(
                status_change = (
                    pl.col('status').ne(pl.col('status').shift(fill_value=20))
                    .over('mmsi')
                ),
                status_previous = pl.col('status').shift().over('mmsi')
            )
            #keep only new status pings
            .filter(pl.col('status_change')==True)
            #drop change col
            .drop('status_change')
        )
        #append to list of lazyframes
        lfs.append(lf)
    except:
        print(f'{file} failed')

#collect all lazyframes
dfs = pl.collect_all(lfs)

In [6]:
#create single pandas dataframe
ais_gdf = (
    #concat dfs
    pl.concat(dfs, how='diagonal_relaxed')
    #sort by vessel and time
    .sort(['mmsi', 'time'])
    #create duration column
    .with_columns(
        status_duration = (pl.col('time').shift(-1) - pl.col('time'))
        .over('mmsi').dt.total_minutes()
    )
    #drop short changes in status between equal statuses
    .with_columns(
        short = ((pl.col('status').shift()==pl.col('status').shift(-1)) & 
                (pl.col('status_duration')<min_duration)).over('mmsi')
    )
    .filter(pl.col('short')!=True)
    #drop repeated same-status messsages
    .with_columns(
        repeat = (pl.col('status')==pl.col('status').shift()).over('mmsi')
    )
    .filter(pl.col('repeat')!=True)
    .drop('repeat')
    #recalculate duration column
    .with_columns(
        status_duration = (pl.col('time').shift(-1) - pl.col('time'))
        .over('mmsi').dt.total_minutes()
    )
    #convert to pandas
    .to_pandas()
)

#convert to geopandas dataframe
ais_gdf = (
    #convert to geodataframe
    gpd.GeoDataFrame(
        ais_gdf,
        geometry=gpd.points_from_xy(ais_gdf.lon, ais_gdf.lat, crs='EPSG:4326')
    )
    #convert to WGS84 pseudo-mercator
    .to_crs(3857)
    #drop old lat lon cols
    .drop(['lat', 'lon'], axis=1)
)

### Port and Dock Data

Locations and descriptions for each dock and port come from the BTS and USACE online databases. 

In [7]:
#load port data
ports_gdf = (
    #read in shape file downloaded from BTS
    gpd.read_file('port data/Principal_Ports/Principal_Ports.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'PORT', #unknown numeric ID - not CBP or UN code
        'FOREIGN_','EXPORTS', 'IMPORTS', 'DOMESTIC' #breadown of total vol (tons)
    ], axis=1)
)
#set col names to pythonic lowercase
ports_gdf.columns = ports_gdf.columns.str.lower()

#load dock data
docks_gdf = (
    #read in shape file downloaded from USACE
    gpd.read_file('port data/Dock/Dock.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'LONGITUDE', 'LATITUDE', #already coded in 'geometry' 
        'LOCATION_D', #text description of dock location
        'STREET_ADD','ZIPCODE', #street address details
        'PSA_NAME', #statistical area name, rarely used
        'COUNTY_NAM', 'COUNTY_FIP', 'CONGRESS', 'CONGRESS_F', #county and congress info
        'MILE', 'BANK', 'LATITUDE1', 'LONGITUDE1', #redundant locaation data
        'OPERATORS', 'OWNERS', #owner info
        'PURPOSE', #long-form text description of dock uses
        'DOCK', #unknown number (not unique to each row/dock)
        'HIGHWAY_NO', 'RAILWAY_NO', 'LOCATION', #redundant location info
        'COMMODITIE', 'CONSTRUCTI','MECHANICAL', 'REMARKS', 'VERTICAL_D', 
        'DEPTH_MIN', 'DEPTH_MAX','BERTHING_L', 'BERTHING_T', 'DECK_HEIGH', 
        'DECK_HEI_1', #these are rarely used stats on construction
        'SERVICE_IN','SERVICE_TE', #rarely used indicators of data entry date 
    ], axis=1)
    #drop duplicates with matching geometries, keeping most common data
    .groupby('geometry').agg(pd.Series.mode).reset_index()
    #rename cols for clarity
    .rename(columns={
        'NAV_UNIT_I':'nav_unit_id',
        'NAV_UNIT_N':'nav_unit_name',
        'FACILITY_T':'facility_type',
        'CITY_OR_TO':'city',
        'STATE_POST':'state'
    })
)
#set col names to pythonic lowercase
docks_gdf.columns = docks_gdf.columns.str.lower()

#coerse back to gdf - groupby appears to have kicked it back to pandas core
docks_gdf = gpd.GeoDataFrame(docks_gdf, geometry='geometry', crs=3857)

### Matching Port and Dock data with AIS Messages

First, we match each of the moored (aka "docked" aka "at berth") AIS messages with the nearest port, then backfill the preceeding AIS status changes with the relevant port info. This allows us to observe which port the vessel was headed towards at any give time. 

A similar operation is performed with each dock. 

In [8]:
start = time.time()
stops_gdf = (
    #filter to only moorings
    ais_gdf[ais_gdf.status == 5]
    #join in nearest port to each ais message
    .sjoin_nearest(ports_gdf, how='left', exclusive=True,
                   #set max distance to 10km
                   max_distance = 10000
                   )
    #drop unneeded cols
    .drop(['index_right', 'total'], axis=1)
    #rename cols for clarity
    .rename({'rank':'port_rank', 'type':'port_type'}, axis=1)
)

#create main df
main_gdf = (
    #merge stops back into AIS data
    ais_gdf.merge(stops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#mark port name to unknown for docking messages not matched within max distance
main_gdf.loc[(main_gdf.status==5) & main_gdf.port_name.isnull(), 'port_name'] = 'unknown'
#backfill port info across vessels, except geometry (normal pandas syntax not supported for gpd geometry)
main_gdf[['port_type','port_name','port_rank']] = (
    main_gdf[['mmsi', 'port_type','port_name','port_rank']].groupby('mmsi').bfill()
)
#drop messages with missing or unknown port info (these vessels did not dock near a US port)
main_gdf = main_gdf[main_gdf['port_name'].notnull() & (main_gdf['port_name']!='unknown')]
#merge port geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_gdf = main_gdf.merge(ports_gdf[['port_name', 'geometry']], 
                          on='port_name', how='left', suffixes=[None, '_port'])
#compute distance from message loc to port loc
main_gdf['port_dist'] = main_gdf['geometry'].distance(main_gdf['geometry_port'])
#create year and month cols for convenience
main_gdf['year'] = main_gdf['time'].dt.year
main_gdf['month'] = main_gdf['time'].dt.strftime('%Y%m')

In [9]:
#add dock info to main df
dockstops_gdf = (
    #filter to only moorings
    main_gdf[main_gdf.status == 5]
    #join in nearest dock to each ais message
    .sjoin_nearest(
        #keep only dock id
        docks_gdf[['nav_unit_id', 'geometry']],
        #max distanance 1km
        max_distance = 1000, 
        how='left')
    #drop unneeded cols
    .drop(['index_right'], axis=1)
)

#merge docks match back into main gdf
main_gdf = (
    #merge stops back into AIS data
    main_gdf.merge(dockstops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#backfill dock info across vessels, except geometry (normal pandas syntax not supported for gpd geometry)
main_gdf[['nav_unit_id']] = (
    main_gdf[['mmsi', 'nav_unit_id']].groupby('mmsi').bfill()
)
#coerse nav_unit_ids to string for merge 
main_gdf['nav_unit_id'] = main_gdf['nav_unit_id'].astype(str)
docks_gdf['nav_unit_id'] = docks_gdf['nav_unit_id'].astype(str)
#merge dock geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_gdf = main_gdf.merge(docks_gdf[['nav_unit_id', 'geometry']], 
                          on='nav_unit_id', how='left', suffixes=[None, '_dock'])
#compute distance from message loc to dock loc
main_gdf['dock_dist'] = main_gdf['geometry'].distance(main_gdf['geometry_dock'])
#rename nav_unit_id to dock_id
main_gdf.rename({'nav_unit_id':'dock_id'}, axis=1, inplace=True)

#add port and dock lat and long to preserve geometries in polars and pandas dfs
main_gdf['port_lat'] = main_gdf.set_geometry('geometry_port').to_crs("EPSG:4326").geometry_port.y
main_gdf['port_lon'] = main_gdf.set_geometry('geometry_port').to_crs("EPSG:4326").geometry_port.x
main_gdf['dock_lat'] = main_gdf.set_geometry('geometry_dock').to_crs("EPSG:4326").geometry_port.y
main_gdf['dock_lon'] = main_gdf.set_geometry('geometry_dock').to_crs("EPSG:4326").geometry_port.x

In [10]:
display(main_gdf.shape)
main_gdf.head()

(867394, 31)

,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,status_previous,status_duration,short,geometry,port_type,port_name,port_rank,geometry_port,port_dist,year,month,dock_id,geometry_dock,dock_dist,port_lat,port_lon,dock_lat,dock_lon
0,205042000,2021-11-27 18:01:51,1.4,135.6,95.0,0.0,DELOS,80.0,9877767.0,336.0,60.0,13.1,80.0,5.0,180.0,False,POINT (-10789383.384 3220133.945),C,"Port of Long Beach, CA",5.0,POINT (-13159021.347 3993886.017),2.492765e+06,2021,202111,0V0U,POINT (-13160065.524 3996249.272),2.494492e+06,33.73957,-118.2095,3.993886e+06,-1.315902e+07
1,205042000,2021-11-27 21:02:31,2.1,150.7,119.0,3.0,DELOS,80.0,9877767.0,336.0,60.0,13.1,80.0,5.0,1880.0,False,POINT (-10745670.446 3209846.487),C,"Port of Long Beach, CA",5.0,POINT (-13159021.347 3993886.017),2.537515e+06,2021,202111,0V0U,POINT (-13160065.524 3996249.272),2.539239e+06,33.73957,-118.2095,3.993886e+06,-1.315902e+07
2,205042000,2021-11-29 04:22:45,3.6,121.9,109.0,0.0,DELOS,80.0,9877767.0,336.0,60.0,20.1,80.0,3.0,534745.0,False,POINT (-10768169.229 3064348.133),C,"Port of Long Beach, CA",5.0,POINT (-13159021.347 3993886.017),2.565193e+06,2021,202111,0V0U,POINT (-13160065.524 3996249.272),2.567023e+06,33.73957,-118.2095,3.993886e+06,-1.315902e+07
3,205042000,2022-12-05 12:48:11,0.7,161.7,0.0,1.0,DELOS,80.0,9877767.0,336.0,60.0,20.3,80.0,0.0,742.0,False,POINT (-13144130.139 3978148.735),C,"Port of Long Beach, CA",5.0,POINT (-13159021.347 3993886.017),2.166587e+04,2022,202212,0V0U,POINT (-13160065.524 3996249.272),2.411568e+04,33.73957,-118.2095,3.993886e+06,-1.315902e+07
4,205042000,2022-12-06 01:10:28,3.1,201.6,205.0,0.0,DELOS,80.0,9877767.0,336.0,60.0,20.3,80.0,3.0,750.0,False,POINT (-13144426.248 3978187.503),C,"Port of Long Beach, CA",5.0,POINT (-13159021.347 3993886.017),2.143502e+04,2022,202212,0V0U,POINT (-13160065.524 3996249.272),2.389172e+04,33.73957,-118.2095,3.993886e+06,-1.315902e+07


## Save data for presentation in dash app

In [11]:
#%%script echo skipping
#convert to polars
main_df = pl.DataFrame(
    main_gdf.drop(['geometry', 'geometry_port', 'geometry_dock'], axis=1)
    )

#save to parquet
main_df.write_parquet('port data/dashboard/main.parquet')

## Performance Metrics and Exploratory Analysis

### Stats for most recent 12 months

The eventual dashboard will present some statistics and visualizations based on data from the most recently available 12 months. 

In [12]:
#get most recent twelve month window that appears in the data
latest_date = main_gdf.time.max()
year_before = latest_date - pd.DateOffset(months=12)

#create ports stats for most recent 12 months
portstats_last12months_gdf = (
    #convert main gdf to polars
    pl.DataFrame(
        main_gdf.drop(['geometry', 'geometry_port', 'geometry_dock'], axis=1)
    )
    #filter to most recent 12 months
    .filter(pl.col('time')>= year_before)
    #ensure sorting
    .sort(['mmsi', 'time'])
    #create row index (for identifying docking events)
    .with_row_index('docking_id')
    .with_columns(
        #create docking event id - NOTE may need to ensure this captures all relevant messages
        docking_id = (
            #keep only docking ids associated with docking messages
            pl.when(pl.col('status')==5)
            .then(pl.col('docking_id'))
            .otherwise(pl.lit(None))
            #backfill over vessel
            .backward_fill().over('mmsi')
        )
    )
    #drop messages not associated with a docking event
    .drop_nulls(subset='docking_id')
    .with_columns(
        #sum anchorage time for each docking event
        time_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().over('docking_id')
        ),
        #get monthly vessels and visits
        vessels = pl.col('mmsi').n_unique().over('port_name', 'month'),
        visits = pl.col('docking_id').n_unique().over('port_name', 'month')
    )
    #aggregate to ports
    .group_by('port_name')
    .agg(
        #keep lat and long
        port_lat = pl.col('port_lat').first(),
        port_lon = pl.col('port_lon').first(),
        #get monthly average of unique vessels seen at each port
        vessels_avg = pl.col('vessels').mean(),
        #get monthly average of vessel visits at each port
        visits_avg = pl.col('visits').mean(),
        #get median time at berth in hours
        time_at_berth_median = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        #get median time at anchor in hours
        time_at_anchor_median = pl.col('time_at_anchor').median()/60,
        #get mean time at anchor in hours
        time_at_anchor_mean = pl.col('time_at_anchor').mean()/60
    )
    #convert to pandas to that geopandas is happy
    .to_pandas()
)
#convert back to geodataframe
portstats_last12months_gdf = (
    gpd.GeoDataFrame(
        portstats_last12months_gdf, 
        geometry=gpd.points_from_xy(portstats_last12months_gdf.port_lon, 
                                    portstats_last12months_gdf.port_lat),
        crs=3857
    )
)

In [107]:
#create docks stats for most recent 12 months
dockstats_last12months_gdf = (
    #convert main gdf to polars
    pl.DataFrame(
        main_gdf.drop(['geometry', 'geometry_port', 'geometry_dock'], axis=1))
    #filter to most recent 12 months
    .filter(pl.col('time')>= year_before)
    #ensure sorting
    .sort(['mmsi', 'time'])
    #create row index (for identifying docking events)
    .with_row_index('docking_id')
    .with_columns(
        #get monthly count of vessels
        vessels = pl.col('mmsi').n_unique().over('dock_id', 'month'),
        #get monthly count of vessel docking events - NOTE this may need revision 
        visits = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
            .n_unique().over('dock_id', 'month')
        ),
        #create docking event id
        docking_id = (
            pl.when(pl.col('status')==5)
            .then(pl.col('docking_id'))
            .otherwise(pl.lit(None))
            .backward_fill().over('mmsi', 'dock_id')
        )
    )
    .with_columns(
        #sum anchorage time for each docking event
        time_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().over('mmsi', 'docking_id')
        )
    )
    #aggregate
    .group_by('dock_id')
    .agg(
        #keep port name, dock lat and long
        dock_lat = pl.col('dock_lat').first(),
        dock_lon = pl.col('dock_lon').first(),
        port_name = pl.col('port_name').mode().first(),
        #get avg vessels and visits per month
        vessels_avg = pl.col('vessels').mean(),
        visits_avg = pl.col('visits').mean(),
        #get median time at berth in hours
        time_at_berth_avg = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        #get median time at anchor in hours
        time_at_anchor_avg = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60
    )
    #convert to pandas to that geopandas is happy
    .to_pandas()
)

#convert back to geodataframe
dockstats_last12months_gdf = (
    gpd.GeoDataFrame(
        dockstats_last12months_gdf, 
        geometry=gpd.points_from_xy(dockstats_last12months_gdf.dock_lon, 
                                    dockstats_last12months_gdf.dock_lat),
        crs=3857
    )
)

### Monthly Stats

In [126]:
monthly_df = (
    #convert to polars
    pl.DataFrame(main_gdf.drop(['geometry', 'geometry_port', 'geometry_dock'], axis=1))
    #agg over ports and months
    .group_by('port_name', 'month')
    .agg(
        #keep lat and long
        lat = pl.col('port_lat').first(),
        lon = pl.col('port_lon').first(),
        #get monthly avg vessels
        vessels_avg = pl.col('mmsi').n_unique(),
        #get average time at berth
        time_at_berth_avg = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        #get average time at anchor
        time_at_anchor_avg = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60
    )
)

## Visualizations

In [119]:
#scatterplot
fig = px.scatter_geo(
    portstats_last12months_gdf,
    lon='port_lon',
    lat='port_lat',
    size='visits_avg',
    color='time_at_berth_median',
    range_color=[0,50],
    hover_name='port_name',
    size_max=20,
    title='Average visits per month & Median Hours at Berth (previous 12 months)',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_berth_median':'Median Hours at Berth'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to average vessel visits per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

In [131]:
#scatterplot
#scatterplot
fig = px.scatter_geo(
    portstats_last12months_gdf,
    lon='port_lon',
    lat='port_lat',
    size='visits_avg',
    color='time_at_berth_median',
    range_color=[0,50],
    hover_name='port_name',
    size_max=20,
    title='Average visits per month & Median Hours at Berth (previous 12 months)',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_berth_median':'Median Hours at Berth'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to average vessel visits per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

In [128]:
#get top 5 ports
top5ports = pl.Series(ports_gdf.sort_values('rank').head().port_name)

#get top 10 ports
top10ports = pl.Series(ports_gdf.sort_values('rank').head(10).port_name)

In [127]:
px.line(
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='vessels_avg', color='port_name',
    title='Vessels per month at Principal Ports',
    labels={'vessels_avg':'Unique Vessels'},
    width=1000,
    height=500 
)

In [129]:
px.line(
    monthly_df
    #restrict to top 10 ports
    .filter(pl.col('port_name').is_in(top10ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='time_at_anchor_avg', color='port_name',
    title='Median Time at Anchor at Principal Ports',
    width=1000,
    height=500 
)

In [130]:
px.line(
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='vessels_avg', color='port_name',
    title='Unique vessels at Anchor at Principal Ports',
    width=1000,
    height=500 
)

In [ ]:
px.line(
    #data with month in dt format
    monthly_df
    #restrict to top 5 ports
    .filter(pl.col('port_name').is_in(top5ports))
    #month in dt format
    .with_columns(pl.col('month').str.strptime(pl.Date, format='%Y%m'))
    .sort(by='month'), 
    #plot specs
    x='month', y='time_at_berth_avg', color='port_name',
    title='Median Time at Berth at Principal Ports',
    width=1000,
    height=500 
)